In [9]:
from datasets import load_dataset
dataset = load_dataset("nirantk/dbpedia-entities-google-palm-gemini-embedding-001-1K", streaming=False, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['_id', 'title', 'text', 'embedding'],
    num_rows: 1000
})

In [28]:
dataset[0]['embedding']['embedding']

[0.036727894,
 -0.0473622,
 -0.0728433,
 -0.039807934,
 0.081355415,
 0.046048794,
 0.0034380844,
 -0.0225606,
 0.036697697,
 0.00088976545,
 -0.047007225,
 0.014228779,
 -0.023968711,
 -0.014010341,
 0.018770965,
 -0.0033208432,
 -0.012913837,
 0.043013558,
 -0.010479255,
 -0.04824307,
 0.015731642,
 0.037504885,
 -0.007806924,
 0.016691392,
 -0.024495406,
 -0.021860661,
 -0.007035921,
 -0.089892715,
 0.037421316,
 0.044830523,
 -0.06405422,
 0.07728542,
 -0.012455702,
 0.0055673467,
 0.00012712895,
 -0.04447241,
 -0.0030322075,
 0.05899769,
 0.017172787,
 0.042557098,
 0.0042247763,
 -0.007824668,
 -0.024594678,
 -0.03853048,
 -0.011352426,
 -0.018860903,
 -0.0125966845,
 0.00024429712,
 -0.0431176,
 -0.048764564,
 0.04549857,
 -0.0332576,
 0.02523635,
 -0.017218545,
 -0.008121888,
 -0.061722696,
 0.06560158,
 -0.0063109105,
 -0.072628066,
 -0.0020522578,
 -0.0067197243,
 0.021857427,
 0.00042296434,
 -0.0013249633,
 -0.03768392,
 -0.06350892,
 -0.054922715,
 0.04602351,
 0.03697769,

In [15]:
import os
os.environ["QDRANT_API_KEY"] = <API-KEY-HERE>

In [19]:
from qdrant_client import QdrantClient, models

client = QdrantClient(
    url="https://a4197291-1236-40e0-bf18-18e8843a05a2.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key=os.getenv("QDRANT_API_KEY")
)

In [20]:
# Create the collection to hold our embeddings
# on_disk=True and the quantization_config are the areas to focus on
collection_name = "google-palm-001"
client.recreate_collection(
    collection_name=f"{collection_name}",
    vectors_config=models.VectorParams(
        size=768,
        distance=models.Distance.DOT,
        on_disk=True,
    ),
    optimizers_config=models.OptimizersConfigDiff(
        default_segment_number=5,
        indexing_threshold=0,
    ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(always_ram=True),
    ),
)

True

In [45]:
points=[
        {
            "id": i,
            "vector": embedding['embedding'],
            "payload": {"text": data["text"], "title": data["title"]}
        }
        for i, (embedding, data) in enumerate(zip(dataset["embedding"], dataset))
    ]

In [47]:
# points[0]

In [50]:
# Assuming 'points' is your list of points to be upserted
for i in range(0, len(points), 10):
    slice_points = points[i:i+10]  # Create a slice of 10 points
    client.upsert(
        collection_name=collection_name,
        points=slice_points
    )

In [51]:
client.update_collection(
    collection_name=f"{collection_name}",
    optimizer_config=models.OptimizersConfigDiff(
        indexing_threshold=20000
    )
)

True

In [52]:
client.get_collection(collection_name=collection_name)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=1000, indexed_vectors_count=0, points_count=1000, segments_count=5, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=True), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=5, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=BinaryQuantization(binary=Binar

In [55]:
client.search(
    collection_name=f"{collection_name}",
    query_vector=dataset[32]['embedding']['embedding'],
    search_params=models.SearchParams(
        quantization=models.QuantizationSearchParams(
            ignore=False,
            rescore=False,
            oversampling=2.0,
        )
    )
)

[ScoredPoint(id=32, version=99, score=0.9999966, payload={'text': 'Senior hockey refers to amateur or semi-professional ice hockey competition.  There are no age restrictions for senior players, who typically consist of those whose junior eligibility has expired.  The top senior amateur teams in Canada compete annually for the Allan Cup. Senior hockey leagues in North America operate under the jurisdiction of Hockey Canada or USA Hockey.  They are not affiliated in any way with professional hockey leagues.', 'title': 'Senior ice hockey'}, vector=None, shard_key=None),
 ScoredPoint(id=577, version=153, score=0.6943921, payload={'text': 'Powys Wenwynwyn or Powys Cyfeiliog was a Welsh kingdom which existed during the high middle ages.', 'title': 'Powys Wenwynwyn'}, vector=None, shard_key=None),
 ScoredPoint(id=625, version=158, score=0.63147604, payload={'text': 'Adrian Delano Dantley (born February 28, 1955) is a retired American basketball player who played 15 seasons in the National Ba